In [1]:
import pandas as pd
import re
import email
from generate_data import generate_train_and_test_data
import pandas as pd
import math
import numpy as np

In [2]:
emails = pd.read_csv('data/emails.csv',quoting=2,header=0)
emails = emails[emails["file"].str.contains('sent').tolist()]
filelist = emails['file'].tolist()
messages = emails['message'].tolist()
msg = email.message_from_string(messages[1])
print(msg)

Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>
Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)
From: phillip.allen@enron.com
To: john.lavorato@enron.com
Subject: Re:
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.

As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the

#### these are the 10 people whose writing styles we will analyze

In [3]:
filter_list=['kay.mann@enron.com','vince.kaminski@enron.com','jeff.dasovich@enron.com',
                 'chris.germany@enron.com','sara.shackleton@enron.com','tana.jones@enron.com',
                'eric.bass@enron.com','matthew.lenhart@enron.com','kate.symes@enron.com','sally.beck@enron.com']

#### the enron emails often contain forwarded messages, or the orginal text in a reply to email. we remove all this data since we are trying to understand a particular person's writing style.
#### we also split the data into train and test. the test data is what we will try and predict on.

In [4]:
df,df_train,df_test=generate_train_and_test_data('data/emails.csv',filter_list,min_words=0)

In [5]:
print(df.shape)
df_summary=df.pivot_table(index="From",values='FormattedMessage',aggfunc="count")
df_2=df.pivot_table(index="From",values=['NumWords','MessageLength'],aggfunc="mean")
df_summary=df_summary.join(df_2,lsuffix='',rsuffix='')
df_summary

(40027, 6)


,FormattedMessage,MessageLength,NumWords
From,,,
chris.germany@enron.com,4560,236.905921,44.401535
eric.bass@enron.com,2678,144.328603,28.281927
jeff.dasovich@enron.com,4784,435.320025,72.006689
kate.symes@enron.com,2617,307.330149,53.588460
kay.mann@enron.com,8163,202.101188,35.149455
matthew.lenhart@enron.com,2545,107.922593,23.260904
sally.beck@enron.com,2461,490.861032,92.376676
sara.shackleton@enron.com,3950,254.070380,42.507342
tana.jones@enron.com,3307,241.613245,43.262474


In [6]:
all_data=df.pivot_table(index="From",values='NumWords',aggfunc="mean")
train_data=df_train.pivot_table(index="From",values='NumWords',aggfunc='mean')
test_data=df_test.pivot_table(index="From",values='NumWords',aggfunc='mean')
df_summary=all_data.join(train_data,lsuffix='All',rsuffix='Train')
df_summary=df_summary.join(test_data,lsuffix='',rsuffix='Test')
df_summary.columns=['All','Train','Test']
df_summary

,All,Train,Test
From,,,
chris.germany@enron.com,44.401535,43.816232,49.656455
eric.bass@enron.com,28.281927,28.867194,23.188406
jeff.dasovich@enron.com,72.006689,72.068885,71.457906
kate.symes@enron.com,53.588460,53.593683,53.543796
kay.mann@enron.com,35.149455,34.767816,38.548544
matthew.lenhart@enron.com,23.260904,23.253673,23.333333
sally.beck@enron.com,92.376676,93.174482,85.094650
sara.shackleton@enron.com,42.507342,42.317634,44.274151
tana.jones@enron.com,43.262474,42.921496,46.247788


In [7]:
all_data=df.pivot_table(index="From",values='MessageLength',aggfunc="mean")
train_data=df_train.pivot_table(index="From",values='MessageLength',aggfunc='mean')
test_data=df_test.pivot_table(index="From",values='MessageLength',aggfunc='mean')
df_summary=all_data.join(train_data,lsuffix='All',rsuffix='Train')
df_summary=df_summary.join(test_data,lsuffix='',rsuffix='Test')
df_summary.columns=['All','Train','Test']
df_summary

,All,Train,Test
From,,,
chris.germany@enron.com,236.905921,232.893736,272.927790
eric.bass@enron.com,144.328603,147.488759,116.826087
jeff.dasovich@enron.com,435.320025,435.813591,430.965092
kate.symes@enron.com,307.330149,307.107981,309.229927
kay.mann@enron.com,202.101188,199.605396,224.330097
matthew.lenhart@enron.com,107.922593,107.573466,111.419913
sally.beck@enron.com,490.861032,495.744364,446.288066
sara.shackleton@enron.com,254.070380,252.721615,266.631854
tana.jones@enron.com,241.613245,239.479784,260.292035


#### take all the text in the training set, and count the total number of words
#### then take the text for each author and count the total number of words by author
#### finally we take the n_most_frequent words 

In [8]:
from nltk.tokenize import sent_tokenize,word_tokenize
from collections import defaultdict
import nltk

#this is the most frequent count
n_most_frequent=50

#define a dictionary to hold the word count for each author
author_subcorpus_count={}
for item in filter_list:
    author_subcorpus_count[item]=defaultdict(int)


all_text=df_train['FormattedMessage'].tolist()
from_list=df_train['From'].tolist()

#word counts for the combined corpus
word_counts=defaultdict(int)

#go through the entire corpus, count words for the combined corpus and for each author
for i,text in enumerate(all_text):
    sentences=sent_tokenize(text.lower())
    for sentence in sentences:
        words=word_tokenize(sentence)
        fdist = nltk.FreqDist(words)
        for word in fdist:
            word_counts[word]+=fdist[word]
            author_subcorpus_count[from_list[i]][word]+=fdist[word]
            
#create a list of most frequent words
#also check what total word count is (to validate data)
freq_list=[]
i=0
totalWords=0
for w in word_counts:
    totalWords+=word_counts[w]
for w in sorted(word_counts, key=word_counts.get, reverse=True):
    if i<n_most_frequent: freq_list.append((w,word_counts[w]))
    i+=1
totalWords

1843025

In [9]:
freq_list

[('.', 91894),
 ('the', 71283),
 (',', 66043),
 ('to', 51610),
 ('i', 42181),
 ('and', 29998),
 ('a', 26967),
 ('of', 24905),
 ('you', 24804),
 ('in', 20779),
 ('for', 19969),
 ('that', 18942),
 ('is', 18413),
 ('on', 15771),
 ('it', 15007),
 ('we', 14029),
 ('this', 13639),
 ('?', 13243),
 ('have', 13240),
 (')', 13030),
 ('be', 12821),
 ('with', 12696),
 ('(', 12492),
 ("'s", 11369),
 ('will', 10800),
 ('me', 9034),
 (':', 8671),
 ('are', 8539),
 ('at', 8228),
 ('if', 7999),
 ('thanks', 7689),
 ("n't", 7556),
 ('as', 7359),
 ('do', 7310),
 ('from', 6502),
 ('can', 6364),
 ('my', 6360),
 ('not', 6347),
 ("''", 6139),
 ('know', 6058),
 ('your', 5868),
 ('would', 5837),
 ('was', 5489),
 ('or', 5416),
 ('!', 5389),
 ('please', 5322),
 ('but', 5187),
 ('-', 5107),
 ('``', 5070),
 ('>', 4939)]

In [10]:
#aggregate total words by author
#ensure it adds up to total words by corpus
totalWordsByAuthor={}
totalWords=0
for author in author_subcorpus_count:
    totalWordsByAuthor[author]=sum(author_subcorpus_count[author][x] for x in author_subcorpus_count[author])
    totalWords+=totalWordsByAuthor[author]

#we compute the mean for the corpus 2 ways
#by corpus - so for example count "the" in the entire corpus/ total words in corpus
#or compute the prob of "the" in each author's corpus and average it
#the 2 results are not that different
frequentWordsCorpusMean={}
frequentWordsCorpusStdDev={}
for word,count in freq_list:
    frequentWordsCorpusMean[word]=(count+0.000001)/totalWords
    frequentWordsCorpusStdDev[word]=0.0


topWordsByAuthor={}
for item in author_subcorpus_count:
    topWordsByAuthor[item]={}
    for word,count in freq_list:
        wc=author_subcorpus_count[item][word]
        wp=(wc+0.000001)/totalWordsByAuthor[item]
        topWordsByAuthor[item][word]=wp
        
frequentWordsMean={}
for word,count in freq_list:
    frequentWordsMean[word]=0.0
    for author in topWordsByAuthor:
        frequentWordsMean[word]+=topWordsByAuthor[author][word]
    frequentWordsMean[word]/=len(topWordsByAuthor)

for word,count in freq_list:
    for author in topWordsByAuthor:
        diff=topWordsByAuthor[author][word]-frequentWordsCorpusMean[word]
        frequentWordsCorpusStdDev[word]+=diff*diff
    frequentWordsCorpusStdDev[word]/=len(topWordsByAuthor)
    frequentWordsCorpusStdDev[word]=math.sqrt(frequentWordsCorpusStdDev[word])
    
#print(frequentWordsCorpusMean)
print(totalWords)

1843025


In [11]:
#calculate zscores
#for each author, calculate the zscore for each of the common words
zScoresByAuthor={}
for author in topWordsByAuthor:
    zScoresByAuthor[author]={}
    for word in frequentWordsCorpusMean:
        zScoresByAuthor[author][word]=(topWordsByAuthor[author][word]-frequentWordsCorpusMean[word])/(frequentWordsCorpusStdDev[word]+0.00001)

In [12]:
#this function calculates the zscore for the test text
#it takes the text and counts the probabilities for common words
#and uses the frequentWordsCorpusMean and frequentWordsCorpusStdDev
def calc_z_score(text,frequentWordsCorpusMean,frequentWordsCorpusStdDev):
    word_counts=defaultdict(int)
    totalWords=0
    sentences=sent_tokenize(text.lower())
    for sentence in sentences:
        words=word_tokenize(sentence)
        fdist = nltk.FreqDist(words)
        for word in frequentWordsCorpusMean:
            if word in fdist:
                word_counts[word]+=fdist[word]
            else:
                word_counts[word]+=0
            totalWords+=fdist[word]
    zScores={}
    for word in word_counts:
        word_dist=(word_counts[word]+0.000001)/(totalWords+0.000001)
        zScores[word]=(word_dist-frequentWordsCorpusMean[word])/frequentWordsCorpusStdDev[word]
    return zScores

In [13]:
def find_email_match(text,frequentWordsCorpusMean,frequentWordsCorpusStdDev):
    scores={}
    min_score=1000000
    min_name=''
    zscores=calc_z_score(text,frequentWordsCorpusMean,frequentWordsCorpusStdDev)
    for author in zScoresByAuthor:
        score=0.0
        for word in zScoresByAuthor[author]:
            score+=abs(zscores[word]-zScoresByAuthor[author][word])
        score/=len(zScoresByAuthor[author])
        scores[author]=score
        if score<min_score:
            min_score=score
            min_name=author
    return min_name,min_score,scores

In [14]:
i=2310
text=df_test.iloc[i]['FormattedMessage']
name=df_test.iloc[i]['From']
print(name)
print(text)
author,min_score,scores=find_email_match(text,frequentWordsCorpusMean,frequentWordsCorpusStdDev)
print(author,min_score)
print(scores)

kate.symes@enron.com
whadup! i'm so happy for you re: the love thing. love is sooooo in the air 
lately. molly just fell head over heels for some cowboy dude evan introduced 
her to. they've been attached at the hips, literally, for the last three 
weeks. and amber spends about four quality hours a day on the phone with 
marky, then cries when she has to hang up. seriously. but your romance sounds 
much more interesting - you absolutely have to keep me posted on how it 
unfolds. and what's the guy's name anyway? you know what's weird is that last 
night after my haircut i met up with molly and her man for dinner. we decided 
to check out the brazen bean, this little swanky cigar bar across the street 
from the blue moon on 21st. (we heard they had a martini happy hour, and we 
felt like pretending we were sophisticated.) so we're walking into the place 
when molly suddenly remembers that she knows the people who live upstairs 
(it's in one of those old rambling victorian houses) so she

In [15]:
def softmax(x):
    e_x=np.exp(x-np.max(x))
    out=e_x/e_x.sum()
    return out

def cross_entropy_loss(predictions, targets, epsilon=1e-12):
    """
    Computes cross entropy between targets (encoded as one-hot vectors)
    and predictions. 
    Input: predictions (N, k) ndarray
           targets (N, k) ndarray        
    Returns: scalar
    """
    predictions = np.clip(predictions, epsilon, 1. - epsilon)
    N = predictions.shape[0]
    ce = -np.sum(targets*np.log(predictions+1e-9))/N
    return ce

def get_probs(text,encoded_classes,frequentWordsCorpusMean,frequentWordsCorpusStdDev):
    zscores=calc_z_score(text,frequentWordsCorpusMean,frequentWordsCorpusStdDev)
    returnMatrix=[0.0 for author in zScoresByAuthor]
    for author in zScoresByAuthor:
        score=0.0
        for word in zScoresByAuthor[author]:
            score+=abs(zscores[word]-zScoresByAuthor[author][word])
        score/=len(zScoresByAuthor[author])
        for i,item in enumerate(encoded_classes):
            if item==author: returnMatrix[i]=score
    returnMatrix=[-x for x in returnMatrix]
    return softmax(returnMatrix),returnMatrix

In [16]:
from sklearn.preprocessing import LabelBinarizer
enc=LabelBinarizer()
enc.fit(filter_list)
print(enc.classes_)
all_text=df_test['FormattedMessage'].tolist()
from_list=df_test['From'].tolist()
y_values=enc.transform(from_list)
y_pred=[]
for text in all_text:
    prob,blah=get_probs(text,enc.classes_,frequentWordsCorpusMean,frequentWordsCorpusStdDev)
    y_pred.append(prob)
    
y_pred=np.array(y_pred)

['chris.germany@enron.com' 'eric.bass@enron.com' 'jeff.dasovich@enron.com'
 'kate.symes@enron.com' 'kay.mann@enron.com' 'matthew.lenhart@enron.com'
 'sally.beck@enron.com' 'sara.shackleton@enron.com' 'tana.jones@enron.com'
 'vince.kaminski@enron.com']


In [17]:
i=2
print(from_list[i])
print(all_text[i])
print(enc.inverse_transform(y_values)[i])
print(y_pred[i])
print(enc.classes_[np.argmax(y_pred[i])])

chris.germany@enron.com
Update

Jerry and I went to East Texas this weekend.  we withdrew $750.00 from the 
Fredonia bank and I'm going to deposit that in the Credit Union.  I also took 
the petty funds in the kitchen drawer - $72.00.

AND we did get $2,000.00 deposited into Dad's account from the Credit Union.

later dudes
chris.germany@enron.com
[0.10965931 0.07548288 0.14431942 0.11564122 0.09627069 0.07071252
 0.09978375 0.08791403 0.08425849 0.11595769]
jeff.dasovich@enron.com


In [18]:
print(cross_entropy_loss(y_values,y_pred))

18.39284038838596
